# EnergyPlus Weather + Simulation Prediction using LSTM

In [5]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


## Load EPW Weather Data

In [6]:
epw = pd.read_csv("Torino_IT-hour.epw", skiprows=8, header=None)
epw.columns = ["Year", "Month", "Day", "Hour", "Minute", "DataSource", "DryBulb", "DewPoint", "RH", "AtmosPressure", 
               "ExtGlobHorRad", "ExtDirNormRad", "ExtDifHorRad", "GlobalHorRad", "DirectNormRad", 
               "DiffuseHorRad", "InfraSky", "WindDir", "WindSpd", "TotalSkyCover", "OpaqueSkyCover", 
               "Visibility", "CeilingHeight", "PresWeatherObs", "PresWeatherCodes", "PrecipWater", "AerosolOptDepth",
               "SnowDepth", "DaysSinceSnow", "Albedo", "LiquidPrecip", "RainRate", "RainDuration", "SnowRate", "SnowDuration"]

weather_df = epw[["DryBulb", "RH", "DirectNormRad", "DiffuseHorRad"]].copy()
weather_df.columns = ["Temperature", "Humidity", "DirectRad", "DiffuseRad"]
weather_df.head()


,Temperature,Humidity,DirectRad,DiffuseRad
0,13.3,37,0,0
1,12.8,39,0,0
2,12.2,38,0,0
3,11.9,38,0,0
4,11.6,38,0,0


## Load Simulation Output

In [8]:
sim_df = pd.read_csv("eplusout.csv", low_memory=False)
sim_df_hourly = sim_df[[col for col in sim_df.columns if "(Hourly)" in col]].copy()
full_df = pd.concat([weather_df, sim_df_hourly.reset_index(drop=True)], axis=1).dropna()
full_df.head()


,Temperature,Humidity,DirectRad,DiffuseRad
0,13.3,37.0,0.0,0.0
1,12.8,39.0,0.0,0.0
2,12.2,38.0,0.0,0.0
3,11.9,38.0,0.0,0.0
4,11.6,38.0,0.0,0.0


## Prepare Data for LSTM Model

[]


[]


[]


❌ هیچ ستونی برای Operative Temperature پیدا نشد.


In [15]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# تعداد ساعات lag (مثلاً 24 یعنی یک روز)
n_lag = 24

# 📥 ویژگی‌ها (features) از داده آب‌و‌هوا
features = full_df[["Temperature", "Humidity", "DirectRad", "DiffuseRad"]].values

# 🎯 هدف مدل (target): گرمایش ساعتی کل ساختمان
target = full_df["DistrictHeating:Facility [J](Hourly)"].values

# ⚖️ نرمال‌سازی ویژگی‌ها و هدف
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)
Y_scaled = scaler_Y.fit_transform(target.reshape(-1, 1))

# 📊 ساخت توالی‌های زمانی برای ورودی به LSTM
X_seq, Y_seq = [], []
for i in range(n_lag, len(X_scaled)):
    X_seq.append(X_scaled[i - n_lag:i])
    Y_seq.append(Y_scaled[i])

# تبدیل به آرایه‌های numpy
X_seq = np.array(X_seq)
Y_seq = np.array(Y_seq)

# 🧪 تقسیم داده‌ها به آموزش و آزمون
split = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
Y_train, Y_test = Y_seq[:split], Y_seq[split:]



IndexError: list index out of range

## Train LSTM Model

In [ ]:
input_layer = Input(shape=(n_lag, X_seq.shape[2]))
lstm_out = LSTM(64, activation="relu")(input_layer)
output = Dense(1)(lstm_out)
model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss="mae")

model.fit(
    X_train, Y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5)],
    verbose=1
)


## Predict and Visualize Results

In [ ]:
predictions = model.predict(X_test)
Y_test_inv = scaler_Y.inverse_transform(Y_test)
predictions_inv = scaler_Y.inverse_transform(predictions)

plt.figure(figsize=(12, 5))
plt.plot(Y_test_inv, label="True")
plt.plot(predictions_inv, label="Predicted")
plt.legend()
plt.title("Zone Operative Temperature Prediction")
plt.xlabel("Hour")
plt.ylabel("Temperature (°C)")
plt.grid()
plt.show()
